In [9]:
%pip install cdsapi

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cdsapi
import os

In [2]:
file_format = "netcdf"  # 'grib' or 'netcdf' (but ONLY netcdf is supported by the code below for mapping and time serie extraction)
folder_out = "/home/jovyan/shared-public/SnowPit/tmp"
downloaded_file = 'ERA5-Land hourly.nc'

In [3]:
# List of years
start_year = 2022
end_year = 2022
years = [str(start_year+i) for i in range(end_year-start_year+1)] 

# List of days
start_day = 1
end_day = 31
days = [str(start_day+i).zfill(2) for i in range(end_day-start_day+1)]

# Month(s) and time(s)
month = '3'
time = '00:00'

# Variables
variables_list = 'snow_depth_water_equivalent'

In [6]:
%%time
downloaded_file = os.path.join(folder_out, downloaded_file)

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'year': years,
        'variable': variables_list,
        'month': month,
        'day': days,
        'time': time,
        'format': file_format,
    },
    downloaded_file)

2025-04-01 20:17:42,203 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-01 20:17:42,203 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-01 20:17:44,312 INFO Request ID is 86c478f6-cdfa-41c9-945b-65f570abd176
2025-04-01 20:17:44,535 INFO status has been updated to accepted
2025-04-01 20:17:47,819 INFO status has been updated to running
2025-04-01 20:18:20,400 INFO status has been updated to successful


738079a1ed41eef894c743498a4bfccb.zip:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

CPU times: user 254 ms, sys: 117 ms, total: 372 ms
Wall time: 52.3 s


'/home/jovyan/shared-public/SnowPit/tmp/ERA5-Land hourly.nc'

In [14]:
import netCDF4 as netcdf
import xarray as xr
import rioxarray as rxr

In [16]:
ds = netcdf.Dataset('/home/jovyan/shared-public/SnowPit/tmp/ERA5-Land_hourly.nc', 'r')
ds

OSError: [Errno -51] NetCDF: Unknown file format: '/home/jovyan/shared-public/SnowPit/tmp/ERA5-Land_hourly.nc'